In [1]:
!pip install tensorflow_addons

In [2]:
import os
ROOT_PATH = os.path.abspath("../..") # "/home/jovyan/ChestXray-14"

In [3]:
import sys
sys.path.append(ROOT_PATH)

In [4]:
import tensorflow as tf

from modules.utils import get_dataset
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler, TensorBoard

2023-04-08 20:24:18.007407: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [5]:
from modules.dataset import LABELS

ROOT_PATH: /home/jovyan/ChestXray-14/experiments/Under_sampling-multilabel_classification


In [6]:
ROOT_PATH = os.path.abspath("../..") # "/home/jovyan/ChestXray-14"
INPUT_PATH = f"{ROOT_PATH}/dataset/ChestXray NIH"
EXPERIMENT_NAME = "under_sampling_5_folds_with_focal_loss_with_model_dropout"

In [7]:
class Dataset:
    INPUT_PATH = INPUT_PATH
    
    def __init__(self, fold_num):
        self.fold_num = fold_num
        self.DATA_PATH = "under_sampling_5_folds_dataset_train_valid_test"
    
    def get_train(self):
        filenames = tf.io.gfile.glob(f'{self.INPUT_PATH}/data/{self.DATA_PATH}/folds/fold{self.fold_num}/train/*.tfrec')
        dataset = get_dataset(filenames)
        return dataset

    def get_valid(self):
        filenames = tf.io.gfile.glob(f'{self.INPUT_PATH}/data/{self.DATA_PATH}/folds/fold{self.fold_num}/valid/*.tfrec')
        dataset = get_dataset(filenames)
        return dataset
    
    def get_test(self):
        filenames = tf.io.gfile.glob(f'{self.INPUT_PATH}/data/{self.DATA_PATH}/test/*.tfrec')
        dataset = get_dataset(filenames)
        return dataset

## tf.keras.mixed_precision for faster training

In [8]:
print(tf.keras.mixed_precision.global_policy())
tf.keras.mixed_precision.set_global_policy('mixed_float16')
print(tf.keras.mixed_precision.global_policy())

<Policy "float32">
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA A100-SXM4-40GB MIG 2g.10gb, compute capability 8.0
<Policy "mixed_float16">


## Model Training

In [9]:
CURRENT_PATH = os.path.abspath("")
CURRENT_PATH

'/home/jovyan/ChestXray-14/experiments/Under_sampling-multilabel_classification'

In [10]:
# Learning rate
def lr_schedule(epoch, learning_rate):
    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate

In [11]:
import datetime

def get_callbacks(NAME, weight_option, fold_num):
    log_dir = f"{CURRENT_PATH}/logs/{EXPERIMENT_NAME}/{NAME}_{weight_option}_FOLD_{fold_num}_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    model_checkpoint_callback = ModelCheckpoint(f'results/models/{EXPERIMENT_NAME}/{NAME}_{weight_option}_FOLD_{fold_num}.h5', monitor='val_loss', mode='min', save_best_only=True)
    early_stop_callback = EarlyStopping(monitor='val_loss', mode="min", patience=20, verbose=1)
    reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', mode="min", factor=0.5, patience=3, verbose=1)
    lr_logging_callback = LearningRateScheduler(lr_schedule)
    
    return model_checkpoint_callback, early_stop_callback, reduce_lr_callback, lr_logging_callback, tensorboard_callback

In [12]:
# Constant variables
NAME = "EfficientNetB0"
EPOCHS = 100
weight_option = None # use `imagenet` or `None` only

In [13]:
import pprint
import numpy as np

def check_label_on_dataset(train_dataset, test_dataset):
    y_list = []
    for x,y in train_dataset:
        # print(y)
        y_list.append(y)
    train_labels = np.vstack(y_list)

    y_list = []
    for x,y in test_dataset:
        # print(y)
        y_list.append(y)
    test_labels = np.vstack(y_list)

    # Print
    pprint.pprint(list(zip(LABELS,train_labels.sum(axis=0))))
    print("Count:", len(train_labels))
    pprint.pprint(list(zip(LABELS, test_labels.sum(axis=0))))
    print("Count: ", len(test_labels))

    print("All:", len(train_labels)+len(test_labels))

In [14]:
from modules.models import Model_under_sampling_dropout_focal_loss

In [15]:
# TODO: Main
import time
from tqdm.notebook import tqdm

NUM_FOLDS = 5
for fold_num in tqdm(range(1, NUM_FOLDS + 1)):
    
    # Callbacks
    (
        model_checkpoint_callback, 
        early_stop_callback, 
        reduce_lr_callback, 
        lr_logging_callback, 
        tensorboard_callback
    ) = get_callbacks(
        NAME, weight_option, fold_num
    )

    # Path for CSV
    path = os.path.join(CURRENT_PATH, "results", "history", EXPERIMENT_NAME, f"{NAME}_{weight_option}_FOLD_{fold_num}")
    os.makedirs(path, exist_ok=True)

    # CSV Logger
    csv_logger = CSVLogger(os.path.join(path, f"history.csv"))

    # Dataset
    dataset = Dataset(fold_num)
    train_dataset, validation_dataset = dataset.get_train(), dataset.get_valid()
    check_label_on_dataset(train_dataset, validation_dataset)

    # Modeling
    transfer_model = tf.keras.applications.efficientnet.EfficientNetB0(
        include_top=False, 
        weights=weight_option,
        input_shape=(224, 224, 3),
        pooling=None
    )

    model = Model_under_sampling_dropout_focal_loss(
        transfer_model,
    )
    model = model.get_model()
    model._name = f"{EXPERIMENT_NAME}_FOLD_{fold_num}"
    model.summary()

    # Record time for training
    start = time.time()
    
    # Visualize
    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=validation_dataset,
        verbose=1, # Show Progress Bar while Traning
        callbacks=[model_checkpoint_callback, csv_logger, early_stop_callback, reduce_lr_callback, lr_logging_callback]
    )
    
    end = time.time()
    print(
        "===== " * 5,
        "Model Traninig for: {:.2f} second(s)".format(end - start),
        "===== " * 5,
        "\n\n"
    )

  0%|          | 0/5 [00:00<?, ?it/s]

2023-04-08 20:24:20.648369: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-08 20:24:21.469947: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8013 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 2g.10gb, pci bus id: 0000:31:00.0, compute capability: 8.0


[('No Finding', 6057),
 ('Atelectasis', 7541),
 ('Consolidation', 2994),
 ('Infiltration', 12735),
 ('Pneumothorax', 3407),
 ('Edema', 1471),
 ('Emphysema', 1607),
 ('Fibrosis', 1099),
 ('Effusion', 8531),
 ('Pneumonia', 909),
 ('Pleural_Thickening', 2190),
 ('Cardiomegaly', 1789),
 ('Nodule', 4053),
 ('Mass', 3750),
 ('Hernia', 139)]
Count: 39244
[('No Finding', 1503),
 ('Atelectasis', 1793),
 ('Consolidation', 779),
 ('Infiltration', 3208),
 ('Pneumothorax', 849),
 ('Edema', 386),
 ('Emphysema', 422),
 ('Fibrosis', 268),
 ('Effusion', 2161),
 ('Pneumonia', 228),
 ('Pleural_Thickening', 532),
 ('Cardiomegaly', 430),
 ('Nodule', 993),
 ('Mass', 891),
 ('Hernia', 42)]
Count:  9811
All: 49055
Model: "under_sampling_5_folds_with_focal_loss_with_model_dropout_FOLD_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                            

2023-04-08 20:24:52.216241: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-04-08 20:24:53.143561: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-08 20:24:53.144812: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-08 20:24:53.144869: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-04-08 20:24:53.146016: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-08 20:24:53.146130: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


2453/2453 [==============================] - 288s 111ms/step - loss: 0.0879 - auc: 0.5001 - f1_score: 0.0499 - precision: 0.1913 - recall: 0.0172 - val_loss: 0.0765 - val_auc: 0.5000 - val_f1_score: 0.0329 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 0.0010
Epoch 2/100
2453/2453 [==============================] - 271s 110ms/step - loss: 0.0772 - auc: 0.5000 - f1_score: 0.0330 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.0765 - val_auc: 0.5000 - val_f1_score: 0.0329 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 0.0010
Epoch 3/100
2453/2453 [==============================] - 268s 109ms/step - loss: 0.0770 - auc: 0.5000 - f1_score: 0.0328 - precision: 0.2000 - recall: 5.1483e-05 - val_loss: 0.0765 - val_auc: 0.5000 - val_f1_score: 0.0329 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 0.0010
Epoch 4/100
2453/2453 [==============================] - ETA: 0s - loss: 0.0769 - auc: 0.4959 - f1_score: 0.0327 - precision: 0.0000e+00 - recall: 0

In [16]:
print("Done")

Done
